![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FFramework+Workflows%2FVertex+AI+AutoML&file=Vertex+AI+Tabular+Workflows+-+TabNet+-+Classification.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/Vertex%20AI%20AutoML/Vertex%20AI%20Tabular%20Workflows%20-%20TabNet%20-%20Classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FFramework%2520Workflows%2FVertex%2520AI%2520AutoML%2FVertex%2520AI%2520Tabular%2520Workflows%2520-%2520TabNet%2520-%2520Classification.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/Vertex%20AI%20AutoML/Vertex%20AI%20Tabular%20Workflows%20-%20TabNet%20-%20Classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Framework%20Workflows/Vertex%20AI%20AutoML/Vertex%20AI%20Tabular%20Workflows%20-%20TabNet%20-%20Classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Vertex AI Tabular Workflows - TabNet - Classification

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('numpy', 'numpy'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [5]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [6]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [7]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [8]:
REGION = 'us-central1'
SERIES = 'frameworks'
EXPERIMENT = 'vertex-ai-tabular-workflows-tabnet'

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

# make this the BigQuery Project / Dataset / Table prefix to store results
BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-', '_')
BQ_TABLE = SERIES
BQ_REGION = REGION[0:2] # use a multi region

Packages

In [ ]:
# import python packages
import os

import numpy as np

# Vertex AI
from google.cloud import aiplatform

# BigQuery
from google.cloud import bigquery

Check Versions of Packages:

In [10]:
aiplatform.__version__

'1.78.0'

Clients

In [13]:
# vertex ai clients
aiplatform.init(project = PROJECT_ID, location = REGION, experiment = SERIES)

# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

Parameters:

In [14]:
DIR = f"files/{EXPERIMENT}"

Environment:

In [15]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## Review Source Data

This is a BigQuery public table of 284,807 credit card transactions classified as fradulant or normal in the column `Class`.
- The data can be researched further at this [Kaggle link](https://www.kaggle.com/mlg-ulb/creditcardfraud).
- Read mode about BigQuery public datasets [here](https://cloud.google.com/bigquery/public-data)

In order protect confidentiality, the original features have been transformed using [principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) into 28 features named `V1, V2, ... V28` (float).  Two descriptive features are provided without transformation by PCA:
- `Time` (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- `Amount` (float) is the value of the transaction
 

### Review BigQuery table:

In [16]:
source_data = bq.query(f'SELECT * FROM `{BQ_SOURCE}` LIMIT 5').to_dataframe()
source_data

Time        V1        V2        V3        V4        V5        V6  \
0   8748.0 -1.070416  0.304517  2.777064  2.154061  0.254450 -0.448529   
1  27074.0  1.165628  0.423671  0.887635  2.740163 -0.338578 -0.142846   
2  28292.0  1.050879  0.053408  1.364590  2.666158 -0.378636  1.382032   
3  28488.0  1.070316  0.079499  1.471856  2.863786 -0.637887  0.858159   
4  31392.0 -3.680953 -4.183581  2.642743  4.263802  4.643286 -0.225053   

         V7        V8        V9  ...       V21       V22       V23       V24  \
0 -0.398691  0.144672  1.070900  ... -0.122032 -0.182351  0.019576  0.626023   
1 -0.055628 -0.015325 -0.213621  ... -0.081184 -0.025694 -0.076609  0.414687   
2 -0.766202  0.486126  0.152611  ...  0.083467  0.624424 -0.157228 -0.240411   
3 -0.687478  0.344146  0.459561  ...  0.048067  0.534713 -0.098645  0.129272   
4 -3.733637  1.273037  0.015661  ...  0.649051  1.054124  0.795528 -0.901314   

        V25       V26       V27       V28  Amount  Class  
0 -0.018518 -0.263291 -0.198600  0.098435     0.0      0  
1  0.631032  0.077322  0.010182  0.019912     0.0      0  
2  0.573061  0.244090  0.063834  0.010981     0.0      0  
3  0.543737  0.242724  0.065070  0.023500     0.0      0  
4 -0.425524  0.511675  0.125419  0.243671     0.0      0  

[5 rows x 31 columns]

In [17]:
source_data.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       Int64
dtype: object

---
## Prepare Data Source

The data preparation includes adding splits for machine learning with a column named `splits` with 80% for training (`TRAIN`), 10% for validation (`VALIDATE`) and 10% for testing (`TEST`). Additionally, a unique identifier was added to each transaction, `transaction_id`. 

>These steps could be done locally at training but are instead done in the source system, BigQuery in this case, which provides several advantages:
>
>-   **Single Source of Truth:** A single data preparation can benefit multiple model training jobs for different architectures or even different team members working on the same model. This ensures consistency and avoids duplication of effort.
>-   **Leverage BigQuery's Power:** BigQuery is highly optimized for large-scale data processing. Performing these operations directly in BigQuery leverages its distributed processing capabilities, making the preparation significantly faster and more efficient than local processing, especially for massive datasets.
>-   **Reduced Data Movement:** Preparing the data in BigQuery reduces the amount of data that needs to be moved out of BigQuery and into the training environment. This minimizes latency and potential bottlenecks associated with data transfer.
>-   **Data Versioning and Reproducibility:** By preparing the splits and unique ID in BigQuery, the specific dataset used for training can be easily tracked and versioned. This enhances the reproducibility of experiments and makes it easier to understand the provenance of the data used in a particular model.
>-   **Simplified Training Pipeline:** The training pipeline becomes simpler because it can directly read pre-split data from BigQuery, eliminating the need for complex splitting logic within the training code.
>-   **Pre-calculated Joins and Features:** BigQuery can be used to pre-calculate joins and engineer new features that are beneficial for the model. This can improve model performance and further reduce the workload during the training phase.
>
>**Further Considerations:**
>
>-   **Data Governance and Security:** BigQuery offers robust data governance and security features. Performing data preparation within BigQuery allows you to maintain control over access and ensure data quality.
>-   **Scalability:** This approach is highly scalable. As your dataset grows, BigQuery can handle the increased workload without requiring significant changes to your data preparation pipeline.
>-   **Cost Optimization:** While moving large amounts of data out of BigQuery can incur costs, performing the preparation steps within BigQuery and only extracting the necessary data for training can often be more cost-effective.
>
>By preparing the data in BigQuery, you create a streamlined, efficient, and reproducible workflow (pipeline) that leverages the strengths of the platform and sets your machine learning models up for success.


### Create/Recall Dataset

In [18]:
dataset = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
dataset.location = BQ_REGION
bq_dataset = bq.create_dataset(dataset, exists_ok = True)

### Create/Recall Table With Preparation For ML

Copy the data from the source while adding columns:
- `transaction_id` as a unique identify for the row
    - Use the `GENERATE_UUID()` function
- `splits` column to randomly assign rows to 'TRAIN", "VALIDATE" and "TEST" groups
    - stratified sampling within the levels of `class` by first assigning row numbers within the levels of `class` then using the with a CASE statment to assign the `splits` level.

In [19]:
job = bq.query(f"""
#CREATE OR REPLACE TABLE
CREATE TABLE IF NOT EXISTS 
    `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS
WITH
    add_id AS (
        SELECT *,
            GENERATE_UUID() transaction_id,
            ROW_NUMBER() OVER (PARTITION BY class ORDER BY RAND()) as rn
            FROM `{BQ_SOURCE}`
    )
SELECT * EXCEPT(rn),
    CASE 
        WHEN rn <= 0.8 * COUNT(*) OVER (PARTITION BY class) THEN 'TRAIN'
        WHEN rn <= 0.9 * COUNT(*) OVER (PARTITION BY class) THEN 'VALIDATE'
        ELSE 'TEST'
    END AS splits
FROM add_id
""")
job.result()
(job.ended-job.started).total_seconds()

0.331

In [20]:
raw_sample = bq.query(f'SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` LIMIT 5').to_dataframe()
raw_sample

Time        V1        V2        V3        V4        V5        V6  \
0  122959.0 -1.327297  0.422904  1.617505  2.291196  2.375055  0.411735   
1  122312.0 -1.988557 -0.720301  0.863204  3.114494  1.847474  0.255881   
2  119592.0  2.139741  0.245651 -2.654856  0.178287  1.336991 -0.724664   
3  114162.0  1.899800  0.368336  0.058981  3.734313  0.106120  0.707141   
4   36601.0  1.092339 -0.063143  1.345504  2.738232 -0.768844  0.857445   

         V7        V8        V9  ...       V23       V24       V25       V26  \
0  0.213517  0.424743 -1.809624  ...  0.192604  0.068281 -0.245725 -0.697654   
1  0.580362 -0.083756 -0.939044  ...  1.564951  0.546312 -0.548531 -0.746620   
2  0.906032 -0.436125 -0.528015  ... -0.216033  0.345316  0.747103  0.700184   
3 -0.472405  0.131502 -0.458634  ...  0.168515  0.674750 -0.104164  0.041509   
4 -0.842873  0.476848  0.501090  ... -0.159378  0.008694  0.568803  0.222004   

        V27       V28  Amount  Class                        transaction_id  \
0  0.038216  0.150059     0.0      0  c97b6e2f-603a-4dbe-9bca-0add881f2084   
1 -0.748016  0.410640     0.0      0  791e403e-d59f-491d-b0b7-d8f8710c07fb   
2 -0.123739 -0.099989     0.0      0  d9e720c5-311d-4cf7-95cb-2256823803ba   
3  0.016028 -0.025852     0.0      0  1e2f3bb7-5c58-4f47-9c3d-d51e6298d6e3   
4  0.032466  0.009505     0.0      0  b1ae958a-0801-453d-a838-b11b4306f559   

   splits  
0    TEST  
1    TEST  
2    TEST  
3    TEST  
4    TEST  

[5 rows x 33 columns]

### Review the number of records for each level of `Class` for each of the data splits:

In [21]:
bq.query(f"""
SELECT splits, class,
    count(*) as count,
    ROUND(count(*) * 100.0 / SUM(count(*)) OVER (PARTITION BY class), 2) AS percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
GROUP BY splits, class
""").to_dataframe()

splits  class   count  percentage
0      TEST      0   28432       10.00
1     TRAIN      0  227452       80.00
2  VALIDATE      0   28431       10.00
3      TEST      1      50       10.16
4     TRAIN      1     393       79.88
5  VALIDATE      1      49        9.96